<title>Decoding</title>

<h1>CHI 2017 Computational Methods Tutorial: Statistical Decoder (Draft)</h1>

<p>
In this part of the tutorial we will explore <em>statistical decoding</em> as a technique for designing intelligent interactive systems.
</p>

<h2>Introduction</h2>

<p>
The problem we are adressing is a user attempting to communciate information over some form of channel. In Human-Computer Interaction (HCI) we typically model a user transmitting <em>information</em> to a computer system.
</p>

The term <em>information</em> in HCI usually refers to characters (for example, typing on a keyboard), words (using speech recognition), commands (for instance by using keyboard shortcuts or touchscreen gestures).

<p>
More formally, we intend to transmit a message $y$ via some for of signal $x$. In a perfect world this would be trivially achieved via a lookup-table. Unfortunately we live in an imperfect world and as a consequence our signal will be perturbed by noise in our neuromuscular system, device sensor imprecision, cognitive errors by the user, etc. Due to this inherent uncertainty it makes sense to model the problem <em>probabilistically</em>.
</p>

We then wish to compute the probability of the message $y$ <em>given</em> the signal $x$. This can be written mathematically as $P(y | x)$.

<p>
The probability $P(y|x)$ is known as a <em>conditional probability</em> and it is (by either the definition of conditional probability or as an axiom of probability):
</p>

\begin{equation}
P(y|x) = \frac{P(x \cap y)}{P(x)}
\end{equation}

<p>
(We are going to assume $P(x)\neq0$ and $P(y)\neq 0$.)
</p>
<p>
The above equation states that the conditional probability of $y$ given $x$ is identical to the ratio of the <em>joint probability</em> of $x$ and $y$ and the probability of $x$. The joint probability of $x$ and $y$ can also be written as $P(x,y)$ or $P(x)p(y)$. 
</p>

<p>
We can rewrite the conditional probability $P(y|x)$ as follows:
</p>

\begin{align}
P(y|x) &= \frac{P(x \cap y)}{P(x)}\\
P(x|y) &= \frac{P(y \cap x)}{P(y)} = \frac{P(x \cap y)}{P(y)}\\
\Rightarrow P(x \cap y) &= P(x|y)P(y) = P(y|x)P(x)\\
\Rightarrow P(y|x) &= \frac{P(x|y)P(y)}{P(x)}\\
\end{align}

<p>
This last expression is known as <em>Bayes' rule</em> (or theorem). Usually we have many possible messages that we wish to decode and $P(y|x)$ will then become the <em>posterior</em> probability distribution, assing a probability to every possible message. Our objective is to compute this posterior probability distribution and select the most probable message.
</p>

<p>
Now, since we are usually only interest in the most probable message $\hat{y}$ we can write:
</p>

\begin{equation}
\hat{y}=\underset{y}{\arg\max}\left[P(y|x)\right]
\end{equation}

<p>
We have already seen that the conditional probability $P(y|x)$ can be written using Bayes' rule:
</p>

\begin{equation}
\hat{y}=\underset{y}{\arg\max}\left[\frac{P(x|y)P(y)}{P(x)}\right]
\end{equation}

<p>
However, as we are only interested in the message that maximises the conditional probability of the message given the signal, the denominator $P(x)$ will be invariant and can therefore be dropped:
</p>

\begin{equation}
\hat{y}=\underset{y}{\arg\max}\left[P(x|y)P(y)\right]
\end{equation}

<p>
$P(x|y)$ is the <em>likelihood</em> of the signal $x$ given a particular hypothesis for what the message $y$ could be. $P(y)$ is the <em>prior</em> probability of the message, that is, without taking any signal into account. For instance, if a system can only recognise two messages, $x_1$ and $x_2$, and both are equally likely in the absence of any additional information, then the prior probability of either $x_1$ or $x_2$ is $0.5$.
</p>

<p>
Identifying the highest probable message $y$ is a <em>search problem</em>. We search by consulting a model of the likelihood of a signal $x$ given a message $y$ under consideration and by consulting a model of the prior probability of a message $y$ without any consideration to any signal. This search will generate <em>hypotheses</em> and these hypotheses will have probabilities assigned to them. Usually, the hypothesis with the highest probability assigned to it is our preferred hypothesis:
</p>

\begin{equation}
\hat{\text{hypothesis}}=\underset{\text{hypotheses}}{\arg\max}\left(\text{likelihood model}\cdot\text{prior model}\right)
\end{equation}

<h2>Sequence Decoding</h2>

<p>
Let us assume we have a simple touchscreen keyboard which can generate $k$ distinct letters $L=\left\{l_1, l_2, \ldots, l_k\right\}$.
</p>

We view our signal as a discrete <em>observation sequence</em>, which, for ease of notation, we will denote $O$. When a user is typing on the touchscreen keyboard we are provided with a touch point coordinate $(x,y)$, where $x$ and $y$ are the horizontal and vertical components of the touch point respectively. A set of $n$ touch points then form $n$ observations:

\begin{equation}
O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}
\end{equation}

Since subscripts are often used to denote specific members of a set, we use a superscript within parantheses to denote the index of an individual observation in an sequence (indexing starts at 1). This means that for instance the sequence $l_1, l_2$ means two letters selected from the set of letters that can be generated by the keyboard (which, since we have defined the keyboard above to generate distinct letters, means $l_1 \neq l_2$). In contrast, $l_1^{(1)},l_1^{(2)}$ means we have generated the <em>same</em> letter $l_1$ twice. Similarly, $l_1^{(1)},l_2^{(2)}$ means we have generated the letter sequence $l_1, l_2$.

We are now given a set of observations $O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}$ and we wish to find a <em>hypothesis</em> for $O$, which is a corresponding letter sequence ${L} = \left\{l^{(1)}, l^{(2)}, \ldots, l^{(m)}\right\}$. Note that the cardinality of $O$ and $L$ are not necessarily identical, that is, it is not neccesarily true that $n = m$ ($|O|=|L|$). There are obviously many possible letter sequences that can match the observation sequence. However, we are only interested in the most <em>probable</em> letter sequence $\hat{L}$, and this can be computed using the expression we derived earlier:

\begin{equation}
\hat{L}=\underset{L}{\arg\max}\left[P(O|L)P(L)\right]
\end{equation}

<p>
$P(L)$ is the prior. What would be a suitable model for the prior probability of a letter sequence? This particular problem is known as <em>statistical language modelling</em> and it is a research area that has received considerable attention. A very simple model is the unigram probability model, which in this case simply assigns a probability to each individual letter $l_i$. The probabilities assigned over the letters $l_1, l_2, \ldots, l_k$ (remember that our keyboard could generate $k$ distinct letters) then must sum to one and this is our prior probability distribution.</p>

$P(O|L)$ is the likelihood of the observation sequence given a particular letter sequence hypothesis. For a touchscreen keyboard this means that we need a model that assigns a probability to an individual $(x,y)$ touch point given a particular letter key hypothesis, that is $P(o_i|l_i)$. A simple way to achieve this is to centre a two-dimensional Gaussian distribution at each key centre and assume each 2D Gaussian has equal variance and that there is no correlation between the horizontal and vertical axes (that is, the covariance matrix is diagonal). To compute our likelihood distribution we need to compute the probability of observing a specific touch point for all possible $k$ letters that can be generated by the keyboard. For example, if our keyboard has two keys, say $A$ and $B$ generating the letters $a$ and $b$ respectively, then our likelihood distribution would need to be computed by computing the probability of the touch point for key $A$ and key $B$. If the touch point is centred at key $A$ the probability will be maximised for $A$ and smaller for $B$ (we assume keys do not overlap spatially).

<p>
The search for the most probable letter sequence given an observation sequence can be modelled as a (discrete-time) <em>Markov chain</em>. A Markov chain assumes a future state is conditional on the previously known state and independent of any other previous or future states.
</p>

<p>
When we decode we attempt to find a token that passes through a sequence of states in such a way that it maximises the probability of its hypothesis. This problem can modelled using what is known as a <em>hidden Markov model</em>. The term <em>hidden</em> refer to the fact that the state sequence is unknown. The parameters of the model <em>are</em> known, including any initial, state transition, and emission probabilities.
</p>

<h2>Statistical Decoding using Token Passing</h2>

<p>
We will now explain how to statistically decode the observation sequence $O$ into the most probable letter sequence $L$. There are several algorithms for learning a hidden Markov model and performing inference. For our particular decoding problem a flexible and efficient method is known as <em>token passing</em>. A particular advantage is that it can efficiently search more complex models and it can be easily parallelised.
</p>

<h3>Token</h3>

<p>
Fundamel to token passing is the notion of a <em>token</em>. A token is a data structure that at a minimum contains the following information: 1) the hypothesis generated so far; and 2) the accumulated probability of the hypothesis. For a touchscreen keyboard the hypothesis generated so far is a letter sequence (a string).
</p>

In [8]:
class Token:
    def __init__(self, hypo, acc_prob):
        self.hypo = hypo
        self.acc_prob = acc_prob

<h3>Observation</h3>

<p>
An observation is simply an $(x,y)$ touch point:
</p>

In [6]:
class Observation:
    def __init__(self, x, y):
        self.x = x
        self.y = y

<h3>Observation Sequence</h3>

<p>
The observation sequence $O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}$ is simply a series of observations. They could be kept in a list but it is often convenient to represent them as a separate class when we later improve the model's performance.
</p>

In [7]:
class ObservationSequence:
    def __init__(self, seq):
        self.seq = seq

<h3>Key</h3>

<p>
A key describes the geometry of the keyboard. In this case our likelihood model is very simple. All we need is information on where to centre the 2D Gaussian, which means we need the centre coordinate of the key, and the letter generated by the key.
</p>

In [5]:
class Key:
    def __init__(self, centre_x, centre_y, letter):
        self.centre_x = centre_x
        self.centre_y = centre_y
        self.letter = letter

<h3>Prior</h3>

<p>
The prior is in this instance the simple unigram probability of the letter. It can be implemented using a simple hash table.
</p>

<p>
TODO: Insert code
</p>

<h3>Likelihood</h3>

<p>
The likelihood is a 2D Gaussian probability density function for the key corresponding to the letter under consideration, evaluated at the coordinate specified by the observation:
</p>

<p>
TODO: Insert code
</p>

<h3>Decoder D1: Single-Key Press Substitution-Only Decoder</h3>

<p>
To decode we create an initial token. We then <em>propagate</em> this token to the first observation in the observation sequence. This means that if we have an observation sequence of length 1 and 26 keys on the keyboard we will propagate 26 tokens. Each of these tokens will then attempt to propagate to the next observation in the observation sequence. However, since the sequence is only of length 1 all the tokens have exausted the observation sequence. Such tokens are known as <em>completed tokens</em>. In this case, decoding will end when there are no more tokens to propagate. We then choose the token with the highest accumulated probability and its associated hypothesis (which in this case will consist of a single letter) will be the output. This decoder can be viewed as a memoryless single-key press decoder.
</p>
<p>
Our decoder <em>substitutes</em> our single observation with a letter and outputs a token with a hypothesis (a letter) and an associated probability. Our initial token will propagate 26 tokens, one token for each possible letter that can be generated by the keyboard. The <em>transitions</em> from the initial token to the tokens modelling the 26 possible letter hypotheses for the first observation model the substitutions. 
</p>
<p>
TODO: Insert state-transition diagram.
</p>
<p>
TODO: Insert code.
</p>

<h3>Decoder D2: Single-Key Press Decoder with Substitution and Deletion</h3>

<p>
Our previous decoder D1 has some limitations. While it will successfully identify the most probable letter given a single touch point on the keyboard, it does not model the possibility of an <em>accidental</em> touch. Our D1 decoder before can only handle one possible transition: substitution of an observation with a letter. However, we can augment this decoder relatively easily by adding another transition from the initial token. This transition will model the hypothesis that the observation does not correspond to a letter and it will thus generate a token which in this case remains an empty string. This transition is known as an $\epsilon$-transition (epsilon-transition).
</p>
<p>Since an $\epsilon$-transition does not correspond to a letter key we cannot compute the likelihood using some form of keyboard model such as our 2D Gaussian model. Instead, we assign a fixed penalty probability for an  $\epsilon$-transition. This penalty is a paramter which needs to be set carefully. If it is too low the model will prefer to delete the touch point rather than to generate a letter. On the other hand, if it is set too high, the model will never ignore an accidental touch point.
<p>
TODO: Insert state-transition diagram.
</p>
<p>
TODO: Insert code.
</p>

<h3>Decoder D3: Substition-Only Decoder</h3>

<p>
Our previous decoders D1 and D2 only work one key press at a time. What if we instead receive a series of touch points, that is $O = \left\{o^{(1)}, o^{(2)}, \ldots, o^{(n)}\right\}$?
</p>

<p>
To be able to decode such a model we again start with a single initial token. The process now works as follows. We take any token at observation index $i \in [0,n)$ and we propagate $k$ tokens to observation index $i+1$. We do this until we are at the last observation and can no longer propagate any tokens. These final tokens represent all hypotheses for the observation sequence. The token with the most probable hypothesis is the most likely letter sequence corresponding to the observation sequence.
</p>

<p>
TODO: Insert state-transition diagram.
</p>
<p>
TODO: Insert code.
</p>

<p>
This process will result in an exponential growth in the number of tokens. At observation index 1 we have propaged $k$ tokens, at observation index 2 we have propaged $k^2$ tokens, and so on.
</p>

<p>To keep the search complexity tractable we observe that many tokens that we are propagated are highly unlikely to contain the most probable hypotheses. These tokens can be filtered out using <em>beam pruning</em>. For every observation index we store the highest accumulated probability we have generated so far for that particular observation. We then impose a condition before we propagate a token: only propagate the token if the token's accumulated probability is within a certain distance to the highest accumulated probability for the next observation. This distance is known as the <em>beam width</em> and it is a parameter that can be used to set the operating point (or trade-off) between accuracy and time when searching for a hypothesis. The narrower the beam the more greedy the search becomes. This means decoding will be faster as less tokens are propagated. However, it also means there is a higher risk that a more probable hypothesis is never found as that particular path might have been eliminated at an early stage in the search.
</p>

<p>
TODO: Insert code.
</p>

<h3>Decoder D4: Decoder with Substitution and Deletion</h3>

<p>
This decoder is similar to decoder D2. We modify our <em>propagate</em> function so that instead of generating $k$ transitions ($k$ tokens), each transition modelling a substituion of an observation with a letter, we generate $k+1$ transitions ($k+1$ tokens). The additional transition (token) models an $\epsilon$-transition. As with the D2 decoder above, $\epsilon$-transitions will need to carry a penalty probability.
</p>

<p>
TODO: Insert state-transition diagram.
</p
<p>
TODO: Insert code.
</p>

<h3>Decoder D5: Decoder with Substitution, Insertion and Deletion</h3>

Decoder D4 can substitute observations into either letters or empty strings ($\epsilon$-transitions). However, it cannot model the fact that a user may have accidentally failed to touch a letter key. To correct this behaviour we need to model <em>insertion</em> transitions.

<p>
To model such insertions we modify our <em>propagate</em> function again. Instead of simply propagating $k+1$ tokens to observation index $i+1$, we also propagate $k$ tokens to observation index $i$, the observation index of the token under consideration:
</p>

<p>
TODO: Insert state-transition diagram.
</p
<p>
TODO: Insert code.
</p>

<p>
Our search space is no longer increasing exponentially, it is infinite. The reason is that the decoding process can stay at observation index $i$ indefinately and keep generating new tokens. Therefore insertions need to carry an insertion probability. If the beam width and the insertion penalty are set correctly, the search becomes tractable again.
</p>

<h2>Training and Evaluating a Model</h2>

<h2>Extensions</h2>

<h2>HCI Applications</h2>